Импортируем нужные библиотеки

In [1]:
import torch
import pytorch_lightning as pl

Импортируем нужные классы

In [2]:
import config
from dataset import DataModule
from model import RankingModel

Задаем сиды, чтобы можно было воспроизводить результаты

In [3]:
torch.random.manual_seed(42)
pl.seed_everything(42)

Seed set to 42


42

Создаем необходимые инстанты: данные, модель, учитель.

In [4]:
dm = DataModule(config.TRAIN_DATA_PATH, config.TEST_DATA_PATH, config.BATCH_SIZE, config.NUM_WORKERS)
model = RankingModel(config.INPUT_SIZE, config.LEARNING_RATE)
trainer = pl.Trainer(
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
    )

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/evlko/Documents/GitHub/VK-MLE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Обучаем и тестируем модель.

In [5]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 960   
1 | fc2  | Linear | 2.1 K 
2 | fc3  | Linear | 33    
--------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 177/177 [00:01<00:00, 161.75it/s, v_num=18]      

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation DataLoader 0: 100%|██████████| 59/59 [00:00<00:00, 405.98it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.09021160751581192
        val_ndcg            0.5097045302391052
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 24/24 [00:00<00:00, 381.23it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.10094600915908813
        test_ndcg  

[{'test_loss': 0.10094600915908813, 'test_ndcg': 0.568679928779602}]

Сохраняем модель

In [6]:
script = model.to_torchscript()
torch.jit.save(script, "model.pt")